In [45]:
pip install py-moneyed

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: C:\Users\55859\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip


In [46]:
# importando as bibliotecas

import pandas as pd
import plotly.express as px

In [47]:
# caminho para pasta no drive

path = 'COLE AQUI O CAMINHO DA SUA PASTA'

In [48]:
# lendo o arquivo

df = pd.read_csv('diarias_prefeitura_total.csv')

In [49]:
# visualização das primeiras linhas

df.head(2)

,web-scraper-order,web-scraper-start-url,EXERCÍCIO,UNIDADE ORÇAMENTARIA,Nº EMPENHO,BENEFICIÁRIO,PERÍODO,DESTINO,MOTIVO,VALOR TOTAL DIÁRIA,SITUAÇÃO EMPENHO
0,1687439061-1,https://transparencia.fortaleza.ce.gov.br/inde...,2023,30.101-SECRETARIA MUNICIPAL DO TURISMO DE FORT...,2.0,ADRIANA ALBUQUERQUE NACHON,18/01/2023 a 25/01/2023,MADRID-EX,PARTICIPAR DA FEIRA INTERNACIONAL DE TURISMO -...,"14.588,00",Pago
1,1687439061-2,https://transparencia.fortaleza.ce.gov.br/inde...,2023,30.101-SECRETARIA MUNICIPAL DO TURISMO DE FORT...,1.0,ALEXANDRE PEREIRA SILVA,18/01/2023 a 25/01/2023,MADRID-EX,PARTICIPAR DA FEIRA INTERNACIONAL DE TURISMO -...,"16.672,00",Pago


In [50]:
# primeira análise dos tipos de dados

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9871 entries, 0 to 9870
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   web-scraper-order      9871 non-null   object 
 1   web-scraper-start-url  9871 non-null   object 
 2   EXERCÍCIO              9871 non-null   int64  
 3   UNIDADE ORÇAMENTARIA   9871 non-null   object 
 4   Nº EMPENHO             9843 non-null   float64
 5   BENEFICIÁRIO           9871 non-null   object 
 6   PERÍODO                9871 non-null   object 
 7   DESTINO                9871 non-null   object 
 8   MOTIVO                 9871 non-null   object 
 9   VALOR TOTAL DIÁRIA     9871 non-null   object 
 10  SITUAÇÃO EMPENHO       9871 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 848.4+ KB


# Limpeza e preparação dos dados

In [51]:
# na coluna de valores, vamos tirar as vírgulas

df['VALOR TOTAL DIÁRIA'] = df['VALOR TOTAL DIÁRIA'].str.replace('.', '').str.replace(',', '.')

C:\Users\55859\AppData\Local\Temp/ipykernel_23616/3898699925.py:3: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



Porém, não podemos trabalhar com valores monetários como pontos flutuantes, pois nenhuma linguagem de programação prevê tipos para dinheiro. Haveria erros por arredondamento, como explica Martin Fowler em https://martinfowler.com/eaaCatalog/money.html

Vamos usar o pacote py-moneyed para resolver esse problema.

In [52]:
from moneyed import Money
from moneyed import Money, BRZ

In [53]:
lista_valores = []
for diaria in df['VALOR TOTAL DIÁRIA']:
    valor = Money(diaria, BRZ)
    lista_valores.append(valor.amount)

In [54]:
df['VALOR_TOTAL_CONVERTIDO'] = pd.Series(lista_valores)

In [55]:
df.head(2)

,web-scraper-order,web-scraper-start-url,EXERCÍCIO,UNIDADE ORÇAMENTARIA,Nº EMPENHO,BENEFICIÁRIO,PERÍODO,DESTINO,MOTIVO,VALOR TOTAL DIÁRIA,SITUAÇÃO EMPENHO,VALOR_TOTAL_CONVERTIDO
0,1687439061-1,https://transparencia.fortaleza.ce.gov.br/inde...,2023,30.101-SECRETARIA MUNICIPAL DO TURISMO DE FORT...,2.0,ADRIANA ALBUQUERQUE NACHON,18/01/2023 a 25/01/2023,MADRID-EX,PARTICIPAR DA FEIRA INTERNACIONAL DE TURISMO -...,14588.00,Pago,14588.00
1,1687439061-2,https://transparencia.fortaleza.ce.gov.br/inde...,2023,30.101-SECRETARIA MUNICIPAL DO TURISMO DE FORT...,1.0,ALEXANDRE PEREIRA SILVA,18/01/2023 a 25/01/2023,MADRID-EX,PARTICIPAR DA FEIRA INTERNACIONAL DE TURISMO -...,16672.00,Pago,16672.00


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9871 entries, 0 to 9870
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   web-scraper-order       9871 non-null   object 
 1   web-scraper-start-url   9871 non-null   object 
 2   EXERCÍCIO               9871 non-null   int64  
 3   UNIDADE ORÇAMENTARIA    9871 non-null   object 
 4   Nº EMPENHO              9843 non-null   float64
 5   BENEFICIÁRIO            9871 non-null   object 
 6   PERÍODO                 9871 non-null   object 
 7   DESTINO                 9871 non-null   object 
 8   MOTIVO                  9871 non-null   object 
 9   VALOR TOTAL DIÁRIA      9871 non-null   object 
 10  SITUAÇÃO EMPENHO        9871 non-null   object 
 11  VALOR_TOTAL_CONVERTIDO  9871 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 925.5+ KB


In [57]:
df['VALOR_TOTAL_CONVERTIDO'][50] - df['VALOR_TOTAL_CONVERTIDO'][20]

Decimal('40.00')

In [58]:
# ajustando coluna com nº de empenho

df['Nº EMPENHO'] = df['Nº EMPENHO'].fillna(0) # O que estiver zero, no final, indicará que antes estava nulo
df['Nº EMPENHO'] = df['Nº EMPENHO'].astype(int)
df['Nº EMPENHO'] = df['Nº EMPENHO'].astype(str)

In [59]:
df = df[['EXERCÍCIO', 'UNIDADE ORÇAMENTARIA', 'Nº EMPENHO', 'BENEFICIÁRIO', 'PERÍODO',
       'DESTINO', 'MOTIVO', 'VALOR_TOTAL_CONVERTIDO', 'SITUAÇÃO EMPENHO']]

In [60]:
# Dividindo de coluna com período da viagem em início e fim

df['INÍCIO_VIAGEM'] = df['PERÍODO'].str.split(pat=' a ', expand=True)[0]
df['FIM_VIAGEM'] = df['PERÍODO'].str.split(pat=' a ', expand=True)[1]

In [61]:
print('O que fizemos acima foi "cortar" a coluna "PERÍODO" em duas colunas.')
print('-----------')
print('Desse valor:', df['PERÍODO'][0])
print('-----------')
print('Agora temos uma coluna para o início da viagem:', df['INÍCIO_VIAGEM'][0])
print('E outra para a data final da viagem:', df['FIM_VIAGEM'][0])

O que fizemos acima foi "cortar" a coluna "PERÍODO" em duas colunas.
-----------
Desse valor: 18/01/2023 a 25/01/2023
-----------
Agora temos uma coluna para o início da viagem: 18/01/2023
E outra para a data final da viagem: 25/01/2023


In [62]:
# transformando as colunas de início e fim da viagem em datetime

df['INÍCIO_VIAGEM'] = pd.to_datetime(df['INÍCIO_VIAGEM'], format='%d/%m/%Y')
df['FIM_VIAGEM'] = pd.to_datetime(df['FIM_VIAGEM'], format='%d/%m/%Y')

# calculando a duração da viagem

df['DURAÇÃO_VIAGEM'] = ((df['FIM_VIAGEM'] - df['INÍCIO_VIAGEM'])/pd.offsets.Day(1)).astype(int)

In [63]:
# criando uma nova coluna com a sigla dos destinos de cada viagem, para saber se o servidor viajou para algum estado brasileiro ou para o exterior

lista_destinos = []
for destino in df['DESTINO']:
    viagens = destino[-2:]
    lista_destinos.append(viagens)

df['SG_DESTINO'] = pd.DataFrame(lista_destinos)

In [64]:
print('O que fizemos aqui foi retirar a sigla do destino e separar em uma coluna diferente.')
print('-----------')
print('Por exemplo... No caso desse destino:', df['DESTINO'][0])
print('-----------')
print('Ficou assim:', df['SG_DESTINO'][0])

O que fizemos aqui foi retirar a sigla do destino e separar em uma coluna diferente.
-----------
Por exemplo... No caso desse destino: MADRID-EX
-----------
Ficou assim: EX


In [65]:
# padronizando coluna de unidades orçamentárias

df['COD_UNIDADE_ORÇAMENTARIA'] = df['UNIDADE ORÇAMENTARIA'].str.split(pat='-', expand=True)[0]

In [66]:
print('E aqui retiramos apenas o código das unidades orçamentárias, uma vez que esse elemento permanece o mesmo, ainda que o nome mude em gestões diferentes')
print('-----------')
print('Por exemplo, nesse caso:', df['UNIDADE ORÇAMENTARIA'][0])
print('-----------')
print('Temos apenas o código', df['COD_UNIDADE_ORÇAMENTARIA'][0])

E aqui retiramos apenas o código das unidades orçamentárias, uma vez que esse elemento permanece o mesmo, ainda que o nome mude em gestões diferentes
-----------
Por exemplo, nesse caso: 30.101-SECRETARIA MUNICIPAL DO TURISMO DE FORTALEZA
-----------
Temos apenas o código 30.101


In [67]:
df.head()

,EXERCÍCIO,UNIDADE ORÇAMENTARIA,Nº EMPENHO,BENEFICIÁRIO,PERÍODO,DESTINO,MOTIVO,VALOR_TOTAL_CONVERTIDO,SITUAÇÃO EMPENHO,INÍCIO_VIAGEM,FIM_VIAGEM,DURAÇÃO_VIAGEM,SG_DESTINO,COD_UNIDADE_ORÇAMENTARIA
0,2023,30.101-SECRETARIA MUNICIPAL DO TURISMO DE FORT...,2,ADRIANA ALBUQUERQUE NACHON,18/01/2023 a 25/01/2023,MADRID-EX,PARTICIPAR DA FEIRA INTERNACIONAL DE TURISMO -...,14588.00,Pago,2023-01-18,2023-01-25,7,EX,30.101
1,2023,30.101-SECRETARIA MUNICIPAL DO TURISMO DE FORT...,1,ALEXANDRE PEREIRA SILVA,18/01/2023 a 25/01/2023,MADRID-EX,PARTICIPAR DA FEIRA INTERNACIONAL DE TURISMO -...,16672.00,Pago,2023-01-18,2023-01-25,7,EX,30.101
2,2023,13.901-FUNDO DE APERFEICOAMENTO DA PROCURADORI...,5,VALERIA MORAES LOPES,05/12/2022 a 08/12/2022,SAO PAULO-SP,"VIAGEM A CIDADE DE SAO PAULO SP, AFIM DE PARTI...",1920.00,Pago,2022-12-05,2022-12-08,3,SP,13.901
3,2023,13.901-FUNDO DE APERFEICOAMENTO DA PROCURADORI...,6,NATERCIA SAMPAIO SIQUEIRA,05/12/2022 a 08/12/2022,SAO PAULO-SP,"VIAGEM A CIDADE DE SAO PAULO SP, AFIM DE PARTI...",1120.00,Pago,2022-12-05,2022-12-08,3,SP,13.901
4,2023,13.901-FUNDO DE APERFEICOAMENTO DA PROCURADORI...,1,PABLO FREIRE ROMAO,05/12/2022 a 08/12/2022,SAO PAULO-SP,PARTICIPACAO DE PROCURADOR MUNICIPAL NO XIX CO...,1120.00,Pago,2022-12-05,2022-12-08,3,SP,13.901


# Análise

In [68]:
# primeiramente, vamos ver a situação de empenho das viagens e o total de diárias em cada categoria

df['SITUAÇÃO EMPENHO'].value_counts()

Pago            9352
Anulado          336
Empenhado         85
Liquidado         70
Não aprovado      28
Name: SITUAÇÃO EMPENHO, dtype: int64

In [69]:
# Agora vamos selecionar apenas as viagens pagas, empenhadas ou liquidadas

df_validos = df[(df['SITUAÇÃO EMPENHO'] == 'Pago') | (df['SITUAÇÃO EMPENHO'] == 'Empenhado') | (df['SITUAÇÃO EMPENHO'] == 'Liquidado')]

In [70]:
# Quantidade de viagens por ano - cai em mais da metade no período da pandemia

df_validos[['EXERCÍCIO', 'DESTINO']].value_counts().groupby('EXERCÍCIO').sum()

EXERCÍCIO
2014    1392
2015    1214
2016    1278
2017    1568
2018    1473
2019    1173
2020     478
2021     309
2022     428
2023     194
dtype: int64

In [71]:
# Quantidade de viagens por ano

viagens_orgao_ano = df_validos[['EXERCÍCIO', 'COD_UNIDADE_ORÇAMENTARIA', 'DESTINO']].value_counts().groupby(['EXERCÍCIO', 'COD_UNIDADE_ORÇAMENTARIA']).sum().reset_index()
viagens_orgao_ano_pivot = viagens_orgao_ano.pivot_table(index='COD_UNIDADE_ORÇAMENTARIA', columns='EXERCÍCIO', values=0).fillna(0)

# Para ver o total de viagens por órgão juntando todos os anos, executar a linha abaixo
# viagens_orgao_ano_pivot['Total'] = viagens_orgao_ano_pivot.apply(lambda x: sum(x), axis=1)

In [72]:
viagens_orgao_ano

,EXERCÍCIO,COD_UNIDADE_ORÇAMENTARIA,0
0,2014,01.101,2
1,2014,11.101,46
2,2014,11.202,5
3,2014,11.203,10
4,2014,13.101,14
...,...,...,...
315,2023,31.901,1
316,2023,31.905,1
317,2023,32.101,2
318,2023,33.101,1


In [73]:
# visualizando os o total de diárias pagas por ano e por órgão (Será um impacto da pandemia na redução de viagens?)

px.line(viagens_orgao_ano, y=0 , x='EXERCÍCIO', color='COD_UNIDADE_ORÇAMENTARIA')

In [74]:
# buscando a unidade orçamentária referente ao código

def busca_unidade(cod):
    return df_validos[df_validos['UNIDADE ORÇAMENTARIA'].str.contains(cod)]['UNIDADE ORÇAMENTARIA'].unique()

In [75]:
busca_unidade('19.203')

array(['19.203-INSTITUTO DE PESOS E MEDIDAS',
       '19.203-INSTITUTO DE PESOS E MEDIDAS DE FORTALEZA'], dtype=object)

In [76]:
print('Instituto de Pesos e Medidas: https://www.fortaleza.ce.gov.br/institucional/a-secretaria-349')

Instituto de Pesos e Medidas: https://www.fortaleza.ce.gov.br/institucional/a-secretaria-349


In [77]:
df_2021_2023 = df_validos[df_validos['EXERCÍCIO'].isin([2021, 2022, 2023])]

In [78]:
df_2021_2023[['EXERCÍCIO', 'SITUAÇÃO EMPENHO']].value_counts()

EXERCÍCIO  SITUAÇÃO EMPENHO
2022       Pago                406
2021       Pago                307
2023       Pago                189
2022       Liquidado            22
2023       Liquidado             4
2021       Empenhado             2
2023       Empenhado             1
dtype: int64

In [79]:
# Quantidade de viagens por ano

viagens_orgao_ano_2021_2023 = df_2021_2023[['EXERCÍCIO', 'COD_UNIDADE_ORÇAMENTARIA', 'DESTINO']].value_counts().groupby(['EXERCÍCIO', 'COD_UNIDADE_ORÇAMENTARIA']).sum().reset_index()
viagens_orgao_ano_2021_2023_pivot = viagens_orgao_ano_2021_2023.pivot_table(index='COD_UNIDADE_ORÇAMENTARIA', columns='EXERCÍCIO', values=0).fillna(0)


# Para ver o total de viagens por órgão juntando todos os anos, executar a linha abaixo
viagens_orgao_ano_2021_2023_pivot['Total'] = viagens_orgao_ano_2021_2023_pivot.apply(lambda x: sum(x), axis=1)
viagens_orgao_ano_2021_2023_pivot.sort_values(2022, ascending=False)

EXERCÍCIO,2021,2022,2023,Total
COD_UNIDADE_ORÇAMENTARIA,,,,
19.203,218.0,105.0,25.0,348.0
24.901,4.0,34.0,11.0,49.0
12.101,8.0,33.0,22.0,63.0
23.901,21.0,30.0,19.0,70.0
11.202,2.0,28.0,4.0,34.0
30.101,15.0,23.0,32.0,70.0
17.101,9.0,18.0,9.0,36.0
11.204,0.0,18.0,0.0,18.0
11.205,0.0,16.0,7.0,23.0


## Observando os dados da secretaria de turismo

In [202]:
funcionarios = df_validos[['EXERCÍCIO', 'BENEFICIÁRIO', 'SG_DESTINO', 'COD_UNIDADE_ORÇAMENTARIA', 'VALOR_TOTAL_CONVERTIDO']]

In [210]:
# quantidade de funcionários com diarias pagas por órgão da prefeitura e ano
funcionarios_unidade_orcamentaria = funcionarios[['EXERCÍCIO', 'COD_UNIDADE_ORÇAMENTARIA', 'BENEFICIÁRIO']].groupby(['EXERCÍCIO', 'COD_UNIDADE_ORÇAMENTARIA']).count().reset_index()

In [213]:
px.line(funcionarios_unidade_orcamentaria, y='BENEFICIÁRIO' , x='EXERCÍCIO', color='COD_UNIDADE_ORÇAMENTARIA')

In [222]:
busca_unidade('30.101')

array(['30.101-SECRETARIA MUNICIPAL DO TURISMO DE FORTALEZA',
       '30.101-SECRETARIA MUNICIPAL DE TURISMO DE FORTALEZA'],
      dtype=object)

In [219]:
funcionarios_unidade_orcamentaria.pivot_table(index='COD_UNIDADE_ORÇAMENTARIA', columns='EXERCÍCIO', aggfunc='sum').fillna(0).sort_values(('BENEFICIÁRIO', 2023), ascending=False)

BENEFICIÁRIO                                      \
EXERCÍCIO                        2014   2015   2016    2017   2018   2019   
COD_UNIDADE_ORÇAMENTARIA                                                    
30.101                           29.0   30.0   23.0    20.0   15.0   41.0   
19.203                          836.0  552.0  761.0  1000.0  903.0  537.0   
12.101                            0.0    0.0    0.0     8.0   19.0   10.0   
23.901                            0.0    0.0   72.0    89.0  117.0  106.0   
11.101                           46.0   49.0   23.0    44.0   49.0   52.0   
24.901                           43.0  107.0   70.0    45.0   34.0   47.0   
17.101                            3.0    4.0   10.0    26.0    8.0    7.0   
13.901                           24.0   19.0   25.0    42.0   27.0   27.0   
11.205                            0.0    3.0    4.0     9.0   11.0   11.0   
15.101                           15.0   15.0    4.0    35.0   40.0   34.0   
25.901                           26.0   23.0   13.0    28.0   42.0   34.0   
11.202                            5.0    4.0    1.0     3.0    2.0    0.0   
26.101                           12.0   16.0    4.0     9.0    6.0    8.0   
13.201                            2.0    3.0   11.0     0.0    0.0    0.0   
31.201                            0.0   12.0    5.0    19.0   19.0   14.0   
23.101                          106.0   75.0    8.0     7.0    0.0   18.0   
19.201                            2.0    6.0    4.0     4.0    9.0   14.0   
19.101                            8.0   11.0   12.0    18.0   16.0   33.0   
32.101                           13.0   11.0    7.0     7.0    7.0   15.0   
13.101                           14.0   16.0   46.0    40.0   17.0    3.0   
17.102                           11.0    3.0    8.0     0.0    1.0    5.0   
28.902                            0.0    0.0    0.0     9.0   24.0   32.0   
14.101                            0.0   14.0    4.0     5.0    1.0    5.0   
31.905                            0.0    0.0    0.0     0.0    0.0    0.0   
28.901                           10.0   22.0   17.0     8.0    4.0    3.0   
31.101                           16.0   41.0   18.0     3.0    9.0   14.0   
34.101                            0.0   12.0   19.0    11.0   12.0   20.0   
33.101                            0.0    0.0    0.0     0.0    0.0    0.0   
18.202                           13.0   14.0    7.0     6.0    5.0    9.0   
18.101                            6.0   45.0   20.0    10.0   11.0   12.0   
16.101                           16.0    0.0    0.0     0.0    0.0    0.0   
31.901                            1.0   11.0    4.0    10.0   15.0    9.0   
35.101                           11.0   12.0    5.0     1.0    0.0    0.0   
35.102                            0.0    3.0    5.0     2.0    0.0    0.0   
35.201                            4.0    0.0    0.0     0.0    0.0    0.0   
41.101                            0.0    0.0    0.0     1.0    0.0    0.0   
42.101                            0.0    0.0    0.0     0.0    0.0    2.0   
44.101                            0.0    2.0    5.0     0.0    0.0    0.0   
45.101                            0.0    0.0    1.0     0.0    0.0    0.0   
01.101                            2.0    0.0    0.0     0.0    0.0    0.0   
31.102                            0.0    0.0    0.0     0.0    0.0    2.0   
29.101                            9.0    5.0    3.0     3.0    0.0    7.0   
28.101                           24.0   21.0   19.0    14.0    5.0    5.0   
27.101                           49.0   47.0   27.0    24.0   17.0   12.0   
25.201                            3.0    0.0    6.0     2.0    2.0    2.0   
19.206                            0.0    0.0    0.0     3.0    1.0    1.0   
18.203                            1.0    2.0    0.0     0.0    4.0    1.0   
18.201                            2.0    0.0    0.0     0.0    0.0    0.0   
18.102                            0.0    0.0    0.0     0.0    5.0    2.0   
13.102               

In [238]:
# viagens_exterior

viagens_exterior = df_validos[df_validos['SG_DESTINO'] == 'EX']

In [244]:
df_viagens_exterior = viagens_exterior[['EXERCÍCIO', 'COD_UNIDADE_ORÇAMENTARIA', 'DESTINO', 'BENEFICIÁRIO', 'DURAÇÃO_VIAGEM']]

In [253]:
df_viagens_exterior_orgao = df_viagens_exterior[['EXERCÍCIO', 'COD_UNIDADE_ORÇAMENTARIA']].value_counts().reset_index()

In [270]:
px.bar(df_viagens_exterior_orgao, x='EXERCÍCIO', y=0, color='COD_UNIDADE_ORÇAMENTARIA')

In [271]:
busca_unidade('24.901')

array(['24.901-FUNDO MUNICIPAL DE EDUCACAO'], dtype=object)

In [277]:
df_viagens_exterior_orgao_pivot = df_viagens_exterior_orgao.pivot_table(index='COD_UNIDADE_ORÇAMENTARIA', columns='EXERCÍCIO', aggfunc='sum').fillna(0)
df_viagens_exterior_orgao_pivot.columns = list(range(2014, 2024, 1))
df_viagens_exterior_orgao_pivot

,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
COD_UNIDADE_ORÇAMENTARIA,,,,,,,,,,
11.101,1.0,13.0,5.0,18.0,20.0,11.0,3.0,1.0,5.0,3.0
11.202,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0
11.205,0.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0,5.0,6.0
12.101,0.0,0.0,0.0,1.0,3.0,0.0,0.0,1.0,5.0,2.0
13.101,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13.901,0.0,1.0,5.0,8.0,3.0,3.0,0.0,0.0,0.0,0.0
15.101,0.0,3.0,0.0,5.0,5.0,3.0,0.0,0.0,2.0,1.0
16.101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17.101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [225]:
funcionarios_turismo = funcionarios[funcionarios['COD_UNIDADE_ORÇAMENTARIA'] == '30.101']

In [229]:
funcionarios_turismo_destino_ano = funcionarios_turismo[['EXERCÍCIO', 'SG_DESTINO']].value_counts().reset_index()

In [235]:
funcionarios_turismo_destino_ano.pivot_table(index='SG_DESTINO', columns='EXERCÍCIO').fillna(0)

0                                               
EXERCÍCIO   2014  2015 2016 2017 2018  2019 2020 2021 2022 2023
SG_DESTINO                                                     
AC          22.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0
AM           0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  2.0
BA           0.0   0.0  0.0  0.0  1.0   0.0  0.0  0.0  0.0  0.0
CE           0.0   0.0  4.0  1.0  0.0   0.0  0.0  1.0  2.0  1.0
DF           3.0  12.0  9.0  7.0  5.0   5.0  1.0  4.0  1.0  6.0
EX           0.0   7.0  6.0  2.0  0.0  13.0  2.0  2.0  5.0  8.0
GO           0.0   0.0  0.0  0.0  2.0   0.0  0.0  0.0  0.0  0.0
MA           0.0   0.0  0.0  0.0  0.0   0.0  0.0  1.0  0.0  2.0
MG           1.0   0.0  0.0  0.0  0.0   1.0  0.0  0.0  0.0  2.0
PA           0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  2.0  0.0
PB           0.0   2.0  0.0  0.0  0.0   2.0  0.0  0.0  0.0  0.0
PE           0.0   0.0  2.0  1.0  0.0   0.0  0.0  1.0  2.0  0.0
PR           0.0   1.0  0.0  0.0  0.0   2.0  0.0  2.0  0.0  2.0
RJ           0.0   1.0  1.0  1.0  0.0   1.0  0.0  0.0  0.0  0.0
RN           0.0   0.0  0.0  0.0  0.0   0.0  0.0  1.0  1.0  2.0
RS           0.0   0.0  0.0  0.0  0.0   2.0  0.0  0.0  3.0  0.0
SC           0.0   1.0  0.0  0.0  0.0   2.0  0.0  0.0  2.0  1.0
SP           3.0   6.0  1.0  8.0  7.0  13.0  2.0  3.0  5.0  6.0